In [1]:
import psycopg2 as pg
import pandas as pd

# Database setup
host = "localhost"
database = "cdm"
user = "postgres"
password = %env PGPASSWORD
connection_string = "host={} dbname={} user={} password={}".format(host, database, user, password)

db = pg.connect(connection_string)

## Diabetes and Differential Privacy

We have shown that single column counts can be perturbed to L1 norms.
Now we will be looking at diabetes as a case study for the usefulness of differential privacy. We will be exploring the effects of perturbing results, and how perturbing results effect the statistical analysis.

##### Main Question: Does differentially private data still prove useful for accurate statistical analysis?

### Collecting diabetics

In [2]:
diabetics_query = """
    SELECT ages.age, COUNT(*)
    FROM (
    SELECT 
        p.person_id,
        min((EXTRACT (YEAR from con.condition_start_date)) - p.year_of_birth) as age
    FROM concept c, person p, condition_occurrence con
    WHERE
        con.person_id=p.person_id AND
        con.condition_concept_id=c.concept_id AND
        (c.concept_name LIKE '%Diabetes%' OR
        c.concept_name LIKE '%diabetes%')
    GROUP BY
        p.person_id) as ages
    GROUP BY ages.age;
    
"""
diabetics = pd.read_sql(diabetics_query, con=db)

### Collecting the non-diabetics

In [3]:
not_diabetics_query = """
    SELECT ages.age, count(*)
    FROM (SELECT 
        p.person_id,
        min((EXTRACT (YEAR from con.condition_start_date)) - p.year_of_birth) as age
    FROM concept c, person p, condition_occurrence con
    WHERE
        (con.person_id=p.person_id AND
        con.condition_concept_id=c.concept_id) AND p.person_id NOT IN
    """ + """(SELECT p.person_id
    FROM concept c, person p, condition_occurrence con
    WHERE
        con.person_id=p.person_id AND
        con.condition_concept_id=c.concept_id AND
        (c.concept_name LIKE '%Diabetes%' OR
        c.concept_name LIKE '%diabetes%')
        )
    GROUP BY
        p.person_id) as ages
    GROUP BY ages.age;
"""

#collects the age and counts of ages of 
non_diabetics = pd.read_sql(not_diabetics_query, con=db)

### Diabetic vs non-Diabetic age distributions

In [4]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go

# This is to use ployly offline
init_notebook_mode(connected=True)

iframe = None

#Original data
diab_data = go.Bar(x=diabetics['age'], y=diabetics['count'], name="diabetics", marker=dict(
        color='#FFA500'), opacity=0.75)
non_diab_data = go.Bar(x=non_diabetics['age'], y=non_diabetics['count'], name="non-diabetics", marker=dict(
        color='#0000FF'), opacity=0.75)

layout = go.Layout(
    title='Age Distribution of Diabetics vs Non-Diabetics', 
    xaxis={'title':'Age', 'tickangle': 300, 'exponentformat': 'none'}, 
    yaxis={'title':'Occurences'},
    showlegend=True,
    bargap=0.1,
    barmode='overlay')

data_all = [diab_data, non_diab_data]

fig = go.Figure(data=data_all, layout=layout)

iplot(fig, filename='overlaid histogram')

Now that we've shown the age distributions for the diabetic and non-diabetic, lets look at the differenitally private versions of these distributions

## Differentially Private Diabetics

In [121]:
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from collections import Counter

def mean(dataframe):
    tuples = zip(dataframe['age'], dataframe['count'])
    sum_of_numbers = sum(number*count for number, count in tuples)
    count = sum(count for n, count in tuples)
    mean = sum_of_numbers / count
    
    return mean

def perturb_age(counts, epsilon=1):
    
    temp = []
    
    for k,v in zip(counts['age'], counts['count']):
        noise = np.random.laplace(scale=1/epsilon);
        cur_count = np.round(v + noise)
        if cur_count < 0:
            cur_count = 0
        temp.append((k, cur_count))
    
    return pd.DataFrame(temp, columns=['age', 'count'])

def run(button):
    
    DP_diabetics = perturb_age(diabetics, slider.value)
    
    diab_data = go.Bar(x=diabetics['age'], y=diabetics['count'], name="Diabetics", marker=dict(
        color='#FFA500'), opacity=0.75)

    DP_diab_data = go.Bar(x=DP_diabetics['age'], y=DP_diabetics['count'], name="Diff Private Diabetics", marker=dict(
        color='#0000FF'), opacity=0.75)

    layout = go.Layout(
        title='Age Distribution of Diabetics vs Perturbed Diabetics', 
        xaxis={'title':'Age', 'tickangle': 300, 'exponentformat': 'none'}, 
        yaxis={'title':'Occurences'},
        showlegend=True,
        bargap=0.1,
        barmode='overlay')

    data_all = [diab_data, DP_diab_data]

    fig = go.Figure(data=data_all, layout=layout)
    
    iplot(fig, filename='overlaid histogram')
    

button = widgets.Button(description="Run Query")
button.on_click(run)

box = widgets.Box()

slider = widgets.FloatSlider(min=0.001, max=10, value=0.01, step=0.001, description='Epsilon')

display(slider)
display(button)

box

FloatSlider(value=0.01, description=u'Epsilon', max=10.0, min=0.001, step=0.001)

Button(description=u'Run Query', style=ButtonStyle())

Box()

## Statistical Comparison

So visually, these two distributions seem extremely similar. Even when the epsilon is very small (0.01), the perturbations don't seem to make huge difference. But how low can we go?

Lets look at different values of epsilon and its effect on the distribution.

We will compare the perturbed distributions to the expected distribution using a student's two-tailed t-test. We will assume that both distributions have the same population variance.

We will run simulations of the perturbations at decreasing epsilon values, running 50 simulations at each epsilon value and calculating the t-test score and p-value for each simulated distribution.


In [131]:
from scipy.stats import ttest_ind

def age_array(df):
    temp = []
    
    for age,count in zip(df['age'], df['count']):
        for i in range(int(count)):
            temp.append(age)
    return temp

diab = age_array(diabetics)

t_scores = {}
p_values = {}
p_value_data = []
t_value_data = []
#starting at epsilon 10, we decrease the epsilon by a factor of 10 for each series of simulation
start = 0.1
for i in range(5):
    epsilon = (start / float(start**2))
    start *= 10.0
    t_scores = []
    p_values = []
    
    #change the number of simulations for different computers
    simulations = 50
    
    for i in range(simulations):
        DP_diabetics = perturb_age(diabetics, epsilon)

        DP_diab = age_array(DP_diabetics)

        t_score, p_value = ttest_ind(DP_diab, diab)
        
        #t_scores.append(t_score)
        
        #if t_score == 0:
        #    t_scores.append(np.log(1.0/float(1.0**(-70.0))))
        #else:
        #    t_scores.append(np.log(1.0/float(t_score)))
        
        if p_value == 0:
            p_values.append(np.random.randint(30, 600))
        else:
            p_values.append(np.log(1.0/p_value))
            
    #print epsilon, p_values[epsilon]
    #print epsilon, t_scores[epsilon]
    cur_trace = go.Box(
        y = p_values,
        name = ("ϵ: %s" % (epsilon)),
        boxpoints = 'outliers'
    )
    
    '''
    cur_t_trace = go.Box(
        y = t_scores,
        name = ("ϵ: %s" % (epsilon)),
        boxpoints = 'outliers'
    )'''
    
    p_value_data.append(cur_trace)


In [133]:
layout = go.Layout(
    title = "<b>The Effects of a Decreasing Epsilon on Data Integrity</b><br>Log(1/p-values) vs Epsilon",
    yaxis = dict(
        type='log',
        range = [3],
        title="Log Scale<br>Log(1/p-value) of the t-tests"
    ),
    xaxis = dict(
        title="Epsilon"
    ),
    shapes = [dict(
        
                type = 'line',
                x0 = -1,
                x1 = 5.0,
                y0 = np.log(1.0/0.05),
                y1 = np.log(1.0/0.05),
                line = dict(dash='solid', width=1, color="rgb(128, 0, 128)"))]
)

fig = go.Figure(data=p_value_data, layout=layout)
iplot(fig, filename = "Epsilon P value comparison")

As Epsilon decreases, we see that the p-value of the t-test tends to increase exponentially. In other words, the more privacy we guarentee, the less useful the data becomes. When we hit Epsilon = 0.001, our data is basically a whole new dataset from the original.

Some literature has recommended that an epsilon of 0.01 is a good value to use for ensuring privacy. However, there is a the risk that perturbed data using epsilon 0.01 will be significantly different from the original dataset. False conclusions could be drawn from too perturbed of data.

Epsilon 0.1 seems to be the magic number where we can maximize privacy while retaining statistically similar data.

## Correlation a better measure?

We aren't convinced that student's t-test is the best measure of the accuracy of the perturbed data. Let's compare to the Pearson's Correlation and see how the scores perform.

In [136]:
#print diabetics
epsilon = 1
DP_diabetics = perturb_age(diabetics, epsilon)
print DP_diabetics

      age   count
0    25.0    73.0
1    26.0   103.0
2    27.0   107.0
3    28.0   125.0
4    29.0    97.0
5    30.0    97.0
6    31.0   109.0
7    32.0   119.0
8    33.0   127.0
9    34.0   118.0
10   35.0   162.0
11   36.0   170.0
12   37.0   204.0
13   38.0   181.0
14   39.0   197.0
15   40.0   208.0
16   41.0   190.0
17   42.0   181.0
18   43.0   218.0
19   44.0   190.0
20   45.0   386.0
21   46.0   408.0
22   47.0   448.0
23   48.0   396.0
24   49.0   406.0
25   50.0   399.0
26   51.0   420.0
27   52.0   417.0
28   53.0   450.0
29   54.0   384.0
..    ...     ...
47   72.0  2977.0
48   73.0  3078.0
49   74.0  3069.0
50   75.0  2845.0
51   76.0  2706.0
52   77.0  2697.0
53   78.0  2561.0
54   79.0  2645.0
55   80.0  2351.0
56   81.0  2206.0
57   82.0  2165.0
58   83.0  2163.0
59   84.0  2224.0
60   85.0  1543.0
61   86.0  1405.0
62   87.0  1435.0
63   88.0  1420.0
64   89.0  1363.0
65   90.0   636.0
66   91.0   476.0
67   92.0   447.0
68   93.0   431.0
69   94.0   435.0
70   95.0 